### This file creates a Huggingface dataset from a dataset with Cityscapes format.

In [ ]:
from huggingface_hub import login
hf_token = "<HF_TOKEN>" # Insert your Huggingface token here
login(hf_token)

In [ ]:
base_path = "<DATASET_BASE_PATH>"
huggingface_push_name = "<NEW_DATASET_HF_NAME>"
use_train_id_mapping = True

image_name_identifier = "_leftImg8bit"
label_name_identifier = "_gtFine_labelIds"

image_extension = ".png"

In [ ]:
import numpy as np
from PIL import Image as PILImage
def convert_label_to_trainid(sample):
    # Create the Cityscapes ID to trainID mapping dictionary
    id_to_trainid = {
        0: 255,
        1: 255,
        2: 255,
        3: 255,
        4: 255,
        5: 255,
        6: 255,
        7: 0,
        8: 1,
        9: 255,
        10: 255,
        11: 2,
        12: 3,
        13: 4,
        14: 255,
        15: 255,
        16: 255,
        17: 5,
        18: 255,
        19: 6,
        20: 7,
        21: 8,
        22: 9,
        23: 10,
        24: 11,
        25: 12,
        26: 13,
        27: 14,
        28: 15,
        29: 255,
        30: 255,
        31: 16,
        32: 17,
        33: 18,
        -1: 255
    }

    label = np.array(sample["label"])
    image = sample["image"]  # Keep as PIL Image
    
    converted_label = np.zeros_like(label)
    
    for original_id, train_id in id_to_trainid.items():
        converted_label[label == original_id] = train_id
    
    converted_label = PILImage.fromarray(converted_label)
    
    return {"image": image, "label": converted_label}

In [ ]:
from datasets import Dataset, DatasetDict, Image
import os
from PIL import Image as PILImage
import numpy as np

# Function to get matching image and label paths
def get_image_label_pairs(image_dir, label_dir):
    image_paths = []
    label_paths = []

    # Walk through the image directory
    for root, _, files in os.walk(image_dir):
        for file in files:
            if file.endswith(image_extension):
                # Get the corresponding label file
                city = os.path.basename(root)
                if image_name_identifier is not None:
                    image_name = file.replace(image_name_identifier, "")
                if label_name_identifier is not None:
                    image_name = image_name.replace(image_extension, f"{label_name_identifier}.png")
                else:
                    image_name = file
                label_file = os.path.join(label_dir, city, image_name)

                # Check if the label file exists
                if os.path.exists(label_file):
                    image_paths.append(os.path.join(root, file))
                    label_paths.append(label_file)

    return image_paths, label_paths

# Define paths to Cityscapes dataset
train_image_dir = f"{base_path}/leftImg8bit/train"
train_label_dir = f"{base_path}/gtFine/train"
val_image_dir = f"{base_path}/leftImg8bit/val"
val_label_dir = f"{base_path}/gtFine/val"

# Get image and label paths for train and val sets
train_image_paths, train_label_paths = get_image_label_pairs(train_image_dir, train_label_dir)
val_image_paths, val_label_paths = get_image_label_pairs(val_image_dir, val_label_dir)

# Create Hugging Face datasets for train and val
if len(train_image_paths) > 0:
    train_dataset = Dataset.from_dict({
        "image": train_image_paths,
        "label": train_label_paths,
    })
    # Cast columns to the appropriate types
    train_dataset = train_dataset.cast_column("image", Image())
    train_dataset = train_dataset.cast_column("label", Image())
    # USE TRAIN_ID TRANSFORMATION
    if use_train_id_mapping:
        train_dataset = train_dataset.map(convert_label_to_trainid, num_proc=4)
    train_dataset = train_dataset.cast_column("label", Image())

if len(val_image_paths) > 0:
    val_dataset = Dataset.from_dict({
        "image": val_image_paths,
        "label": val_label_paths,
    })
    val_dataset = val_dataset.cast_column("image", Image())
    val_dataset = val_dataset.cast_column("label", Image())
    if use_train_id_mapping:
        val_dataset = val_dataset.map(convert_label_to_trainid, num_proc=4)
    val_dataset = val_dataset.cast_column("label", Image())


# Wrap the datasets in a DatasetDict
if len(val_image_paths) > 0 and len(train_image_paths) > 0:
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "val": val_dataset,  # Use "validation" as the key
    })
elif len(val_image_paths) > 0:
    dataset_dict = DatasetDict({
        "val": val_dataset,  # Use "validation" as the key
    })
elif len(train_image_paths) > 0:
    dataset_dict = DatasetDict({
        "train": train_dataset,
    })
else: raise Exception

# Print dataset info
print(dataset_dict)

# Save the dataset locally (optional)
#dataset_dict.save_to_disk("cityscapes_dataset")

# Push to Hugging Face Hub (optional)
dataset_dict.push_to_hub(huggingface_push_name, private=True)

In [ ]:
from datasets import load_dataset
ds = load_dataset(huggingface_push_name)

## Test if everything is alright

In [ ]:
ds["train"][2]["label"]

In [ ]:
len(ds["train"])

In [ ]:
np.unique(np.array(ds["val"][2]["label"]))

In [ ]:
ds["val"][2]["image"]